<a href="https://colab.research.google.com/github/meti-94/BERT-QA/blob/main/colbert%20final%20answers%20retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers -q
!pip uninstall -y tensorflow -q
!pip install fuzzywuzzy[speedup] -q
!pip install networkx -q
!pip install pytorch-crf -q
!pip install pattern -q
import nltk
nltk.download('omw-1.4')

     |████████████████████████████████| 5.8 MB 12.1 MB/s 
     |████████████████████████████████| 182 kB 13.7 MB/s 
     |████████████████████████████████| 7.6 MB 62.5 MB/s 
     |████████████████████████████████| 174 kB 18.3 MB/s 
     |████████████████████████████████| 2.2 MB 69.3 MB/s 
     |████████████████████████████████| 22.2 MB 1.4 MB/s 
     |████████████████████████████████| 88 kB 8.2 MB/s 
     |████████████████████████████████| 81 kB 10.7 MB/s 
     |████████████████████████████████| 5.6 MB 71.7 MB/s 
     |████████████████████████████████| 5.6 MB 66.9 MB/s 
     |████████████████████████████████| 348 kB 70.6 MB/s 
     |████████████████████████████████| 100 kB 9.1 MB/s 
     |████████████████████████████████| 4.0 MB 67.3 MB/s 


[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [2]:
!git clone https://github.com/meti-94/BERT-QA.git

Cloning into 'BERT-QA'...
remote: Enumerating objects: 626, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 626 (delta 42), reused 63 (delta 22), pack-reused 531
Receiving objects: 100% (626/626), 117.36 MiB | 18.24 MiB/s, done.
Resolving deltas: 100% (277/277), done.


In [3]:
%cd /content/BERT-QA/src
# model types : [MultiDepthNodeEdgeDetector, BertLSTMCRF, BertCNN, NodeEdgeDetector]
# cross validation: [True, False]
!python train.py NodeEdgeDetector False

/content/BERT-QA/src
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /bert-base-uncased/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:filelock:Attempting to acquire lock 139755293032016 on /root/.cache/huggingface/hub/models--bert-base-uncased/blobs/45a2321a7ecfdaaf60a6c1fd7f5463994cc8907d.lock
DEBUG:filelock:Lock 139755293032016 acquired on /root/.cache/huggingface/hub/models--bert-base-uncased/blobs/45a2321a7ecfdaaf60a6c1fd7f5463994cc8907d.lock
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /bert-base-uncased/resolve/main/config.json HTTP/1.1" 200 570
Downloading: 100% 570/570 [00:00<00:00, 567kB/s]
DEBUG:filelock:Attempting to release lock 139755293032016 on /root/.cache/huggingface/hub/models--bert-base-uncased/blobs/45a2321a7ecfdaaf60a6c1fd7f5463994cc8907d.lock
DEBUG:filelock:Lock 13975529

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp /content/drive/MyDrive/reverb_wikipedia_tuples-1.1.txt /content/BERT-QA/data/

In [4]:
!wget http://reverb.cs.washington.edu/reverb_wikipedia_tuples-1.1.txt.gz
!gzip -d reverb_wikipedia_tuples-1.1.txt.gz
!cp reverb_wikipedia_tuples-1.1.txt /content/BERT-QA/data/

--2022-12-13 07:05:18--  http://reverb.cs.washington.edu/reverb_wikipedia_tuples-1.1.txt.gz
Resolving reverb.cs.washington.edu (reverb.cs.washington.edu)... 128.208.3.120, 2607:4000:200:12::78
Connecting to reverb.cs.washington.edu (reverb.cs.washington.edu)|128.208.3.120|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28489259 (27M) [application/x-gzip]
Saving to: ‘reverb_wikipedia_tuples-1.1.txt.gz’

reverb_wikipedia_tu 100%[===================>]  27.17M  11.7MB/s    in 2.3s    

2022-12-13 07:05:21 (11.7 MB/s) - ‘reverb_wikipedia_tuples-1.1.txt.gz’ saved [28489259/28489259]



In [5]:
!pip install python-terrier -q
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_colbert.git -q
import sys

COLAB='google.colab' in sys.modules

try:
  import faiss
  faiss.get_num_gpus()
except:
  if COLAB:
    print('Installing faiss-gpu from pip ')
    !pip install faiss-gpu==1.6.3 -q
  else:
    print('Installing faiss-gpu via Conda')
    !conda install -c pytorch faiss-gpu -q

import faiss
assert faiss.get_num_gpus() > 0


     |████████████████████████████████| 102 kB 16.5 MB/s 
     |████████████████████████████████| 1.5 MB 71.1 MB/s 
     |████████████████████████████████| 69 kB 6.4 MB/s 
     |████████████████████████████████| 311 kB 70.2 MB/s 
     |████████████████████████████████| 46 kB 4.3 MB/s 
     |████████████████████████████████| 286 kB 71.5 MB/s 
     |████████████████████████████████| 45 kB 3.4 MB/s 
     |████████████████████████████████| 72 kB 1.4 MB/s 
     |████████████████████████████████| 1.2 MB 59.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 133 kB 76.7 MB/s 
     |████████████████████████████████| 293 kB 78.9 MB/s 
     |████████████████████████████████| 52 kB 297 kB/s 
     |████████████████████████████████| 16.5 MB 21.6 MB/s 
     |████████████████████████████████| 209 kB 78.6 MB/s 
     |████████████████████████████████| 147 kB 81.4 MB/s 
     |███████

In [11]:
!rm -rf /content/colbertindex
!python evaluation.py NN

Streaming output truncated to the last 5000 lines.
  Volatile False
  Transient False
  Native False
  Interface False
  Abstract False
  Strict False

DEBUG:jnius.reflect:
meth: toString
  sig: ()Ljava/lang/String;
  Public True
  Private False
  Protected False
  Static False
  Final False
  Synchronized False
  Volatile False
  Transient False
  Native False
  Interface False
  Abstract False
  Strict False

DEBUG:jnius.reflect:
meth: hashCode
  sig: ()I
  Public True
  Private False
  Protected False
  Static False
  Final False
  Synchronized False
  Volatile False
  Transient False
  Native True
  Interface False
  Abstract False
  Strict False

DEBUG:jnius.reflect:
meth: getClass
  sig: ()Ljava/lang/Class;
  Public True
  Private False
  Protected False
  Static False
  Final True
  Synchronized False
  Volatile False
  Transient False
  Native True
  Interface False
  Abstract False
  Strict False

DEBUG:jnius.reflect:
meth: clone
  sig: ()Ljava/lang/Object;
  Public False
  Pr